<a href="https://colab.research.google.com/github/Denis2054/Transformers-for-NLP-and-Computer-Vision-3rd-Edition/blob/main/Chapter08/Fine_tuned_classification_WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tuning classification example
Copyright 2023, Denis Rothman MIT License

OpenAI fine-tunes an ada classifier to distinguish between the two sports: Baseball and Hockey.

This notebook is a modified version of   
[OpenAI's Fine-tuned classification notebook](https://github.com/openai/openai-cookbook/blob/main/examples/Fine-tuned_classification.ipynb)

Denis Rothman has modified parts of this notebook for educational purposes (sections, comments, and code). The main sections titles are those in Fine_tuning_GPT_3.ipynb in the GitHub repository. 2 sections were added:

2.1.Measuring the model's accuracy  
3.1.Generalization

**Note**: Y*ou will have to run the fine-tuning process to create your model and then replace the model names in this notebook with your model name.*

# Installing OpenAI

In [1]:
try:
  import openai
except:
  !pip install openai==0.28 -qq
  import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)

#Comment this cell if you want to enter your key manually.
#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

Mounted at /content/drive


In [3]:
#(2) Enter your manually by
# replacing API_KEY by your key.
#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

# 1.Preparing the dataset

Retrieving the dataset
 The newsgroup dataset can be loaded using sklearn.

In [4]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

First we will look at the data itself:

In [5]:
print(sports_dataset['data'][0])

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously
doubt they will sell out until the end of the season.)

-- 
Doug Bank                       Private Systems Division
dougb@ecs.comm.mot.com          Motorola Communications Sector
dougb@nwu.edu       

In [6]:
sports_dataset.target_names[sports_dataset['target'][0]]

'rec.sport.baseball'

In [7]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

Total examples: 1197, Baseball examples: 597, Hockey examples: 600


One sample from the baseball category can be seen above. It is an email to a mailing list. We can observe that we have 1197 examples in total, which are evenly split between the two sports.

## 1.1. Preparing the data in JSON

We transform the dataset into a pandas dataframe, with a column for prompt and completion. The prompt contains the email from the mailing list, and the completion is a name of the sport, either hockey or baseball. For demonstration purposes only and speed of fine-tuning we take only 300 examples. In a real use case the more examples the better the performance.

In [8]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df.head()

,prompt,completion
0,From: dougb@comm.mot.com (Doug Bank)\nSubject:...,baseball
1,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
2,From: rudy@netcom.com (Rudy Wade)\nSubject: Re...,baseball
3,From: monack@helium.gas.uug.arizona.edu (david...,hockey
4,Subject: Let it be Known\nFrom: <ISSBTL@BYUVM....,baseball


Both baseball and hockey are single tokens. We save the dataset as a jsonl file.

In [9]:
df.to_json("sport2.jsonl", orient='records', lines=True)

##  1.2. Converting the data to JSONL
We can now use a data preparation tool which will suggest a few improvements to our dataset before fine-tuning. Before launching the tool we update the openai library to ensure we're using the latest data preparation tool. We additionally specify `-q` which auto-accepts all suggestions.

In [10]:
!openai tools fine_tunes.prepare_data -f sport2.jsonl -q

Analyzing...

- Your file contains 1197 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 11 examples that are very long. These are rows: [134, 200, 281, 320, 404, 595, 704, 838, 1113, 1139, 1174]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts e

The tool helpfully suggests a few improvements to the dataset and splits the dataset into training and validation set.

A suffix between a prompt and a completion is necessary to tell the model that the input text has stopped, and that it now needs to predict the class. Since we use the same separator in each example, the model is able to learn that it is meant to predict either baseball or hockey following the separator.
A whitespace prefix in completions is useful, as most word tokens are tokenized with a space prefix.
The tool also recognized that this is likely a classification task, so it suggested to split the dataset into training and validation datasets. This will allow us to easily measure expected performance on new data.

creating the training and validation files on OpenAI

In [22]:
# Upload the training file
training_file_response = openai.File.create(
    file=open("/content/sport2_prepared_train.jsonl", "rb"),
    purpose='fine-tune'
)

# Extract the training file ID
training_file_id = training_file_response["id"]
print("Training file ID:", training_file_id)

# Upload the validation file
# Make sure to replace '/path/to/validation_file.jsonl' with your actual validation file path
validation_file_response = openai.File.create(
    file=open("/content/sport2_prepared_valid.jsonl", "rb"),
    purpose='fine-tune'
)

# Extract the validation file ID
validation_file_id = validation_file_response["id"]
print("Validation file ID:", validation_file_id)

Training file ID: file-wFcDc1GtAXsO2RpsttGMp1wJ
Validation file ID: file-nV8TFAJR1ys4SzQvXuxgNjIM


# 2.Fine-tuning an original model


In [24]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.FineTuningJob.create(training_file=training_file_id, validation_file=validation_file_id, model="babbage-002")

<FineTuningJob fine_tuning.job id=ftjob-hWKthHiHv1jB0a9ATkBM2Kza at 0x798ef563cc20> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-hWKthHiHv1jB0a9ATkBM2Kza",
  "model": "babbage-002",
  "created_at": 1705002888,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-h2Kjmcir4wyGtqq1mJALLGIb",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-nV8TFAJR1ys4SzQvXuxgNjIM",
  "training_file": "file-wFcDc1GtAXsO2RpsttGMp1wJ",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": null
}

When the model has been trained,  we can  we can use it for an inference task.

# 3.Running the fine-tuned GPT-3 model

We will now run the model for a classification task

Let's first retrieve the 'sport2_prepared_valid.jsonl' using the file ID we using during the training process for the validation file.

In [14]:
file_id="file-mCLhXE6D5RW7rRzQwEbzCFfo"
file_details = openai.File.retrieve(file_id)
print(file_details)

{
  "object": "file",
  "id": "file-mCLhXE6D5RW7rRzQwEbzCFfo",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 387349,
  "created_at": 1693235843,
  "status": "processed",
  "status_details": null
}


In [26]:
# Create the fine-tuning job
fine_tuning_response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="babbage-002"
)

In [28]:
# Capture the fine-tuning job ID
fine_tuning_job_id = fine_tuning_response["id"]
print("Fine-tuning job ID:", fine_tuning_job_id)

Fine-tuning job ID: ftjob-FwhihzwSRh2KRngBwWzhWLdM


In [34]:
job_details = openai.FineTuningJob.retrieve(fine_tuning_job_id)
print(job_details)

{
  "object": "fine_tuning.job",
  "id": "ftjob-FwhihzwSRh2KRngBwWzhWLdM",
  "model": "babbage-002",
  "created_at": 1705003143,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-h2Kjmcir4wyGtqq1mJALLGIb",
  "result_files": [],
  "status": "running",
  "validation_file": "file-nV8TFAJR1ys4SzQvXuxgNjIM",
  "training_file": "file-wFcDc1GtAXsO2RpsttGMp1wJ",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": null,
  "error": null
}


In [35]:
model=job_details.get("fine_tuned_model")
print(model)

None


In [15]:
test = pd.read_json('sport2_prepared_valid.jsonl', lines=True)
test.head(10)

,prompt,completion
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
1,From: smorris@venus.lerc.nasa.gov (Ron Morris ...,hockey
2,From: golchowy@alchemy.chem.utoronto.ca (Geral...,hockey
3,From: krattige@hpcc01.corp.hp.com (Kim Krattig...,baseball
4,From: warped@cs.montana.edu (Doug Dolven)\nSub...,baseball
5,From: jerry@sheldev.shel.isc-br.com (Gerald La...,baseball
6,From: kime@mongoose.torolab.ibm.com (Edward Ki...,baseball
7,From: cmk@athena.mit.edu (Charles M Kozierok)\...,baseball
8,From: pkortela@snakemail.hut.fi (Petteri Korte...,hockey
9,From: gak@wrs.com (Richard Stueven)\nSubject: ...,hockey


We can now call the model to get the predictions.

We need to use the same separator following the prompt which we used during fine-tuning. In this case it is `\n\n###\n\n`. Since we're concerned with classification, we want the temperature to be as low as possible, and we only require one token completion to determine the prediction of the model.

In [16]:
# text to classify
text_content = test['prompt'][0]
#print(text_content)

In [17]:
ft_model="ft:babbage-002:personal::7sYWloYn"

In [18]:
# Construct the prompt
prompt_text = f"For the following email, please classify by selecting the appropriate option:\n\n{text_content}\n\nOptions:\n1. hockey\n2. baseball\n\nAnswer:"

# Use the completion API to get a prediction
response = openai.Completion.create(
  model=ft_model,
  prompt=prompt_text,
  max_tokens=3  # Limit the response to a few tokens to avoid lengthy outputs
)
print(response.choices[0].text.strip())


hockey

________________________________


## 3.1. Generalization
Interestingly, our fine-tuned classifier is quite versatile. Despite being trained on emails to different mailing lists, it also successfully predicts tweets.

In [19]:
text_content = """Thank you to the
@Canes
 and all you amazing Caniacs that have been so supportive! You guys are some of the best fans in the NHL without a doubt! Really excited to start this new chapter in my career with the
@DetroitRedWings
 !!"""

# Construct the prompt and run the task
prompt_text = f"For the following text, please classify by selecting the appropriate option:\n\n{text_content}\n\nOptions:\n1. hockey\n2. baseball\n\nAnswer:"
res = openai.Completion.create(model="davinci-002", prompt=prompt_text, max_tokens=1, temperature=0, logprobs=2)
res['choices'][0]['text']

' hockey'